## 필요한 라이브러리 불러오기

In [123]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd
from selenium.webdriver import ActionChains

## 인기 앱/게임 정보 및 리뷰 크롤링

In [104]:
def pop_crawler(link, app_cnt):
  # 크롬 드라이버(크롬 브라우저 컨트롤) 다운로드 https://sites.google.com/a/chromium.org/chromedriver/home
  driver = webdriver.Chrome('./config/chromedriver')
  driver.get(pop_link)

  #앱 컨테이너
  common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div'

  # 수집하고자 하는 데이터에 맞게 데이터프레임 생성
  PlaystoreData = pd.DataFrame(columns=['Collection Date', 'App Name', 'App Developer', 'App Category', 'App Pop Rank', 'App Download', 'App Review Count', 'App Charge', 'App Limit'])
  PlaystoreReviewData = pd.DataFrame(columns=['Collection Date', 'App Name', 'Comment Date', 'User', 'Rating', 'Helpful', 'Comment', 'Answer'])

  # 스크롤 내리는 수 (스크롤 한번 당 리뷰 40개씩 증가), 3의 경우 총 리뷰 120개 수집
  scroll_cnt = 3

  for i in range(app_cnt):
    if i >= 50: #50개 이상의 앱 리뷰를 수집하는 경우 필요합니다. 다만 현재 기준은 최대 100위까지이므로 이후 페이지까지 갱신하고자 하면 코드를 수정해야 합니다.
      #드라이버 새로고침 - 새 화면이 뜰 수 있도록 도움
      driver.refresh()
      time.sleep(1)
      
      #첫 페이지의 49번째 컨테이너를 선택합니다. 첫 페이지는 무조건 나오므로 첫 페이지의 가장 마지막 요소를 기준으로 합니다.
      move_under = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div[49]')
      action = ActionChains(driver)
      action.move_to_element(move_under).perform()
      time.sleep(1)

      #페이지 확장 시 앱 컨테이너 변경됨
      common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/c-wiz'
      game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i-50+1)))
      game_path.click()
    else:
      game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i+1)))
      game_path.click()
    
    os.makedirs('result', exist_ok=True) 
    time.sleep(1)

    #앱 정보 가져오기
    Collection_Date = datetime.today().strftime("%Y-%m-%d")
    time.sleep(1)
    App_Name = driver.find_element_by_css_selector('.AHFaub > span').text
    App_Rank = i+1

    try:
      App_Review_cnt = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[2]/c-wiz/span/span[1]').text
      App_Developer = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[1]/a').text
      App_Category = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[2]/a').text

    except:
      App_Review_cnt = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[2]/c-wiz/span/span[1]').text
      App_Developer = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[1]').text
      App_Category = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[2]/a').text
      pass

    # 맨 아래로 스크롤
    more_btn = driver.find_element_by_xpath("//*[@id='fcxH9b']/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[2]")
    action = ActionChains(driver)
    action.move_to_element(more_btn).perform()
  # 하단 앱 정보 가져오기
    try:
      App_Download = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[3]/span/div/span').text
      App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text  
    except:                                              
      App_Download = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[3]/span/div/span').text
      App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text  
      pass

    App_Charge = 'Free'

    # 앱 정보 데이터프레임에 추가
    PlaystoreData = PlaystoreData.append({
      'Collection Date': Collection_Date,
      'App Name': App_Name,
      'App Developer': App_Developer,
      'App Category': App_Category,
      'App Pop Rank': App_Rank,
      'App Download': App_Download,
      'App Review Count': App_Review_cnt,
      'App Charge': App_Charge,
      'App Limit': App_Limit
    }, ignore_index=True)
    driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span').click()
    
    for i in range(scroll_cnt):
      time.sleep(2)     
      scroll = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[3]/div/div[2]/div/div/main/div/div[2]') # 스크롤해야 할 위치 찾기
      action = ActionChains(driver)
      action.move_to_element(scroll).perform() # 스크롤 실행

    print(App_Name,'의 데이터를 수집하고 있습니다')
    # 리뷰 컨테이너 가져오기 
    reviews = driver.find_elements_by_xpath('//*[@jsname="fk8dgd"]//div[@class="d15Mdf bAhLNe"]')
    print(' %d 개의 리뷰가 존재합니다.' % len(reviews))
    
    # 리뷰 데이터 가져오기
    for review in reviews:
      #BeautifulSoup 객체 가져오기
      soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')  

      # 리뷰한 유저 이름
      user = soup.find(class_='X43Kjb').text

      # 별점
      rating = int(soup.find('div', role='img').get('aria-label').replace('별표 5개 만점에', '').replace('개를 받았습니다.', '').strip())

      # 리뷰 날짜
      comment_date = soup.find(class_='p2TkOb').text
      comment_date = datetime.strptime(comment_date, '%Y년 %m월 %d일')
      comment_date = comment_date.strftime('%Y-%m-%d')

      # 유용함
      helpful = soup.find(class_='jUL89d y92BAb').text
      if not helpful:
        helpful = 0
      
      # 리뷰 내역
      comment = soup.find('span', jsname='fbQN7e').text
      if not comment:
        comment = soup.find('span', jsname='bN97Pc').text
      
      # 리뷰에 대한 개발사 답변
      answer = None
      a_div = soup.find('div', class_='LVQB0b')
      if a_div:
        answer = a_div.text.replace('\n', ' ')
      
      # 리뷰 데이터 데이터프레임에 추가
      PlaystoreReviewData = PlaystoreReviewData.append({
        'App Name' : App_Name, 
        'Comment Date' : comment_date, 
        'User' : user, 
        'Rating' : rating, 
        'Helpful' : helpful, 
        'Comment': comment, 
        'Answer' : answer
      }, ignore_index=True)

    print(App_Name,'은(는) ', App_Rank, '위입니다. 이 앱은 리뷰 수집이 끝났습니다.')

    driver.back()
    driver.back()

  driver.stop_client()
  driver.close()

## 최고 매출 게임 정보 및 리뷰 크롤링

In [127]:
def best_crawler(link, app_cnt):
  # 크롬 드라이버(크롬 브라우저 컨트롤) 다운로드 https://sites.google.com/a/chromium.org/chromedriver/home
  driver = webdriver.Chrome('./config/chromedriver')
  driver.get(best_link)

  #앱 컨테이너
  common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div'

  # 수집하고자 하는 데이터에 맞게 데이터프레임 생성
  PlaystoreBest = pd.DataFrame(columns=['Collection Date', 'App Name', 'App Developer', 'App Category', 'App Best Rank', 'App Download', 'App Review Count', 'App Charge', 'App Limit'])

  for i in range(app_cnt):
    if i >= 50: #50개 이상의 앱 리뷰를 수집하는 경우 필요합니다. 다만 현재 기준은 최대 100위까지이므로 이후 페이지까지 갱신하고자 하면 코드를 수정해야 합니다.
      #드라이버 새로고침 - 새 화면이 뜰 수 있도록 도움
      driver.refresh()
      time.sleep(1)
      
      move_under = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div[49]')
      action = ActionChains(driver)
      action.move_to_element(move_under).perform()
      time.sleep(1)

      #페이지 확장 시 앱 컨테이너 변경됨
      common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/c-wiz'
      game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i-50+1)))
      game_path.click()
    else:
      game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i+1)))
      game_path.click()
    
    os.makedirs('result', exist_ok=True) 
    time.sleep(1)

    
    #앱 정보 가져오기
    Collection_Date = datetime.today().strftime("%Y-%m-%d")
    time.sleep(1)
    App_Name = driver.find_element_by_css_selector('.AHFaub > span').text
    App_Rank = i+1

    print(App_Name,'의 데이터를 수집하고 있습니다')  
    
    try:
      App_Review_cnt = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[2]/c-wiz/span/span[1]').text
      App_Developer = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[1]/a').text
      App_Category = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[2]/a').text

    except:
      App_Review_cnt = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[2]/c-wiz/span/span[1]').text
      App_Developer = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[1]').text
      App_Category = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[2]/a').text
      pass

    # 맨 아래로 스크롤
    more_btn = driver.find_element_by_xpath("//*[@id='fcxH9b']/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[2]")
    action = ActionChains(driver)
    action.move_to_element(more_btn).perform()
  # 하단 앱 정보 가져오기
    try:
      App_Download = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[3]/span/div/span').text
    except:
      App_Download = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[3]/span/div/span').text
      pass
    
    try:
      App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text
    except:
      try:
        App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[7]/span/div/span/div[1]').text
      except:
        App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text
        pass
      pass
        
    # 앱 정보 데이터프레임에 추가
    PlaystoreBest = PlaystoreBest.append({
      'Collection Date': Collection_Date,
      'App Name': App_Name,
      'App Developer': App_Developer,
      'App Category': App_Category,
      'App Best Rank': App_Rank,
      'App Download': App_Download,
      'App Review Count': App_Review_cnt,
      'App Limit': App_Limit
    }, ignore_index=True)
  
    print(App_Name,'은(는) ', App_Rank, '위입니다. 앱 정보 수집을 마쳤습니다.')

    driver.back()

  driver.stop_client()
  driver.close()

## 메인 코드

In [133]:
if __name__ == "__main__":
  # 수집하고자 하는 앱 개수 (인기 앱/게임의 경우 한 페이지당 최대 50위까지 가능)
  app_cnt = 100

  # 인기 앱/게임 페이지 링크
  pop_link = 'https://play.google.com/store/apps/collection/cluster?clp=0g4cChoKFHRvcHNlbGxpbmdfZnJlZV9HQU1FEAcYAw%3D%3D:S:ANO1ljJ_Y5U&gsr=Ch_SDhwKGgoUdG9wc2VsbGluZ19mcmVlX0dBTUUQBxgD:S:ANO1ljL4b8c'
  # pop_crawler(pop_link, app_cnt)

  # 최고 매출 게임 페이지 링크
  best_link = 'https://play.google.com/store/apps/collection/cluster?clp=0g4YChYKEHRvcGdyb3NzaW5nX0dBTUUQBxgD:S:ANO1ljLhYwQ&gsr=ChvSDhgKFgoQdG9wZ3Jvc3NpbmdfR0FNRRAHGAM%3D:S:ANO1ljIKta8'
  best_crawler(best_link, app_cnt)

오딘: 발할라 라이징 의 데이터를 수집하고 있습니다
오딘: 발할라 라이징 은(는)  1 위입니다. 앱 정보 수집을 마쳤습니다.
리니지M 의 데이터를 수집하고 있습니다
리니지M 은(는)  2 위입니다. 앱 정보 수집을 마쳤습니다.
리니지2M 의 데이터를 수집하고 있습니다
리니지2M 은(는)  3 위입니다. 앱 정보 수집을 마쳤습니다.


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=91.0.4472.124)


In [ ]:
# PlaystoreData

,Collection Date,App Name,App Developer,App Category,App Pop Rank,App Download,App Review Count,App Charge,App Limit
0,2021-07-04,오딘: 발할라 라이징,Kakao Games Corp.,롤플레잉,1,"500,000+","57,420",Free,18세 이상
1,2021-07-04,히어로즈 테일즈,37 Mobile Games,롤플레잉,2,"100,000+","1,609",Free,12세 이상


In [56]:
# PlaystoreReviewData

,App Name,Comment Date,User,Rating,Helpful,Comment,Answer
0,오딘: 발할라 라이징,2021-07-02,Kyis Lee,5,126,개인적인 생각입니다. mmorpg는 사냥터(주로 필드)를 붐비게하면 성공이라고 생...,None
1,오딘: 발할라 라이징,2021-07-02,CH P,1,200,"그래픽은 좋으나 그게 끝임 게임. 여타 게임과 같은 자동이동, 버튼만 누르면 퉤 완...",None
2,오딘: 발할라 라이징,2021-07-02,이은정,1,143,게임 접속이 너무 어렵습니다. 유저 제한을 하던가 어떻게 방책을 세워줘야지... 열...,None
3,오딘: 발할라 라이징,2021-07-02,민윤지,5,18,게임은 재미 있어요 불편한것만 개선해주시면 만족합니다. 채팅창에 알림이 너무 많아서...,None
4,오딘: 발할라 라이징,2021-07-02,김경민,1,24,30레벨 이상 키워본 경험자들은 알거다 이건 걍 그래픽 감성 게임이다. 컨트롤X 사...,None
...,...,...,...,...,...,...,...
9644,나무 블록 퍼즐 - 무료 클래식 블록 퍼즐 게임,2021-03-04,mi J,5,1,은근 어려운 두뇌게임 이네요 할수록 재미집니다,None
9645,나무 블록 퍼즐 - 무료 클래식 블록 퍼즐 게임,2021-03-07,최영숙,5,9,재미 있고 도전 하고싶은 마음이 너무 많이 드네요,None
9646,나무 블록 퍼즐 - 무료 클래식 블록 퍼즐 게임,2021-05-26,은서리,5,0,나름 시간보내기 좋은 게임,None
9647,나무 블록 퍼즐 - 무료 클래식 블록 퍼즐 게임,2021-03-22,이형기,5,3,광고도 적고 재미있어요,None


In [132]:
PlaystoreBest

NameError: name 'PlaystoreBest' is not defined

## 데이터셋 저장하기

In [59]:
# CSV파일로 저장하기
# PlaystoreReviewData.to_csv('./playstorereviewdata.csv', encoding='utf-8-sig', index=False)
# PlaystoreData.to_csv('./playstoredata.csv', encoding='utf-8-sig', index=False)

In [43]:
# JSON파일로 저장하기
# PlaystoreReviewData.to_json('./playstorereviewdata.json', orient='table')
# PlaystoreReviewData.to_json('./playstoredata.json', orient='table')

,Collection Date,App Name,App Developer,App Category,App Rank,App Download,App Charge,App Limit
0,2021-07-01,오딘: 발할라 라이징,Kakao Games Corp.,롤플레잉,1,"100,000+",0,18세 이상
1,2021-07-01,히어로즈 테일즈,37 Mobile Games,롤플레잉,2,"50,000+",0,12세 이상
2,2021-07-01,비홍지경,withHUG,롤플레잉,3,"100,000+",0,12세 이상
3,2021-07-01,쿠키런: 오븐브레이크 - 쿠키들의 러닝 게임!,Devsisters Corporation,아케이드,4,"10,000,000+",0,만 3세 이상
4,2021-07-01,마이리틀포레스트,Gamepub,시뮬레이션,5,"500,000+",0,전체이용가
5,2021-07-01,X2: 이클립스,Netease Games Global,롤플레잉,6,"10,000+",0,12세 이상
6,2021-07-01,블라스트M,Longtu Korea Inc.,롤플레잉,7,"100,000+",0,12세 이상
7,2021-07-01,"Fidget Toys 3D - Fidget Cube, AntiStress & Calm",Fidget Dev,시뮬레이션,8,"10,000,000+",0,전체이용가
8,2021-07-01,Kaiju Run,HOMA GAMES,캐주얼 게임,9,"1,000,000+",0,전체이용가
9,2021-07-01,마스터 카운트: 군중 클래시 & 스틱맨 달리기 게임,Freeplay Inc,액션,10,"10,000,000+",0,전체이용가
